<a href="https://colab.research.google.com/github/Valent0296/allvent.github.io/blob/master/Test_tesi_ResNet_PE2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#IMPORTO LA CARTELLA MATH_LOADER CON RELATIVI CONTENUTI DALLA LOCATION DRIVE
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/MyDrive/math_loader/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import math
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = nn.Sequential(
    models.resnet18(pretrained=True).conv1,
    models.resnet18(pretrained=True).bn1,
    models.resnet18(pretrained=True).relu,
    models.resnet18(pretrained=True).maxpool,
    models.resnet18(pretrained=True).layer1,
    models.resnet18(pretrained=True).layer2,
    models.resnet18(pretrained=True).layer3,
    models.resnet18(pretrained=True).layer4,
    models.resnet18(pretrained=True).avgpool
).to(device)

model_ft.add_module("0", nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)))
model_ft.add_module("8", nn.AdaptiveAvgPool2d(output_size=(6,6)))
model_ft.get_submodule("7").get_submodule("0").get_submodule("downsample").add_module("0", nn.Conv2d(256, 256, kernel_size=(1, 1), stride=(2, 2), bias=False))
model_ft.get_submodule("7").get_submodule("0").get_submodule("downsample").add_module("1", nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
model_ft.get_submodule("7").get_submodule("0").add_module("conv2", nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
model_ft.get_submodule("7").get_submodule("0").add_module("bn2", nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
model_ft.get_submodule("7").get_submodule("1").add_module("conv1", nn.Conv2d( 256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
model_ft.get_submodule("7").get_submodule("1").add_module("conv2", nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
model_ft.get_submodule("7").get_submodule("1").add_module("bn2", nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
for idx, name in enumerate(model_ft.modules()):
  name.requires_grad_(True)
print(model_ft)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, 

In [ ]:
%cd /content/drive/MyDrive/math_dataloader/
import data as d
import language as l

/content/drive/.shortcut-targets-by-id/1yEYXJtFSq_AJGXoLWG9r6AGr2yS9iSIt/math_dataloader


In [ ]:
filename = ["data/" + fi for fi in ["train.txt", "test.txt", "validation.txt"]]

training = True
height = 160
width = 1600
dataset=[]
for i in range(3):
  training = True if i == 1 else False
  dataset.append(d.ExprDataset("data/", filename[i], training, height,
                          width, geometric_var=0.05,
                          photometric_var=0.25))
n_tok = len(dataset[0].tokens)

In [ ]:
train_dataloader = DataLoader(dataset[0], batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset[1], batch_size=64, shuffle=True)
val_dataloader = DataLoader(dataset[2], batch_size=64, shuffle=True)
"""for images, codes, paths in test_dataloader:
  print(codes.size())"""
#VEDERE PERCHé ALCUNE IMMAGINI NON VENGONO CARICATE, E CAPIRE PERCHÈ LE DIMENSIONI DANNO PROBLEMI

'for images, codes, paths in test_dataloader:\n  print(codes.size())'

In [ ]:
##################################
# Positional Encoding for Tokens #
##################################
class PositionalEncoding( nn.Module ):

  def __init__( self, d_model: int=256, dropout: float=0.1, maxlen: int=10 ):
    super(PositionalEncoding, self).__init__()
    den = torch.exp( torch.arange( 0, d_model, 2 ) * ( -math.log(10000) / d_model ) )
    pos = torch.arange(0, maxlen).reshape( maxlen, 1 )
    self.pe = torch.zeros( (maxlen, d_model) ).to(device)
    self.pe[:, 0::2] = torch.sin(pos*den)
    self.pe[:, 1::2] = torch.cos(pos*den)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("pos_enc", self.pe)

  def forward(self, ccn_embed: Tensor):
    return self.dropout( ccn_embed + self.pe )

######################################
# Positional Encoding for Embeddings #
######################################
class PositionalEncoding2D( nn.Module ):

  def __init__( self, d_model: int=256, dropout: float=0.1, height: int=10, width: int=10 ):
    super(PositionalEncoding2D, self).__init__()
    """
    :param d_model: dimension of the model
    :param height: height of the positions
    :param width: width of the positions
    :return: d_model*height*width position matrix
    """
    if d_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(d_model))
    self.pe = torch.zeros(d_model, height, width).to(device)
    # Each dimension use half of d_model
    d_model = int(d_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) *
                         -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    self.pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    self.pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    self.pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    self.pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("pos_enc", self.pe)

  def forward(self, ccn_embed: Tensor):
    return self.dropout( ccn_embed + self.pe)


###################
# Input embedding #
###################
class SampleEmbedding( nn.Module ):

  def __init__(self, d_emb: int=256):
      super(SampleEmbedding, self).__init__()
      self.ccn = model_ft

  def forward(self, blocks: Tensor):
    return torch.utils.checkpoint.checkpoint_sequential(self.ccn, 5, blocks).to(device)

##################
# Token Embedder #
##################
class TokenEmbedding(nn.Module):

  def __init__(self, vocab_size: int, d_model: int):
    super(TokenEmbedding, self).__init__()
    self.embedder = nn.Embedding(vocab_size, d_model)
    self.emb_size = d_model

  def forward(self, tokens: Tensor):
    return self.embedder( tokens.long() ) * math.sqrt(self.emb_size).to(device)

###########
# Masking #
###########
def generate_square_subsequent_mask(sz):
    #generetes a triangular upper matrix for attention masking
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_pad_mask( matrix: torch.tensor, pad_token: int) -> torch.tensor:
    # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
    # [False, False, False, True, True, True]
    return (matrix == pad_token)

############
# Accuracy #
############
def batch_accuracy_calculate( pred: Tensor, y: Tensor):

  corr = 0
  tot = 0
  for p, label in zip(pred[:] ,y[:]):


    for tok_p, tok_l in zip(p[:], label[:]):


      if tok_p.item() != 3 and tok_l.item() != 3:
        corr += (tok_p == tok_l).type(torch.int).sum().item()
        tot += 1


      else:
        return [corr, tot]

  return [corr, tot]

In [ ]:
##############
# Tranformer #
##############
class TransformerModel(nn.Module):

  def __init__(self, ntokens: int, d_mod: int, d_hid: int ,n_head: int=8, dropout: float=0.5):
      super().__init__()
      self.model_type = "Transformer"
      self.nhead = n_head
      self.d_model = d_mod
      self.src_embedding = SampleEmbedding(d_mod)
      self.pos_encod = PositionalEncoding2D(d_mod, dropout, 6, 6)
      self.tgt_encod = PositionalEncoding(d_mod, dropout, maxlen=20)
      self.tgt_embed = nn.Embedding(n_tok,d_mod)
      self.flat = nn.Flatten(start_dim=2)
      encoder_layer = nn.TransformerEncoderLayer( d_mod, n_head, d_hid, batch_first=True )
      self.transformer_encoder = nn.TransformerEncoder(encoder_layer, n_head)
      decoder_layer = nn.TransformerDecoderLayer( d_mod, n_head, dropout=dropout, batch_first=True)
      self.transformer_decoder = nn.TransformerDecoder(decoder_layer, n_head )
      self.lin_soft_stack = nn.Sequential(
          nn.Linear(d_mod, ntokens),
          nn.Softmax(dim=1)
      )

  def forward(self, src: Tensor, tgt: Tensor, tgt_mask: Tensor, pad_mask: Tensor) -> Tensor:

    src_emb = []
    for i in range(src.size(0)):
      src_emb.append(self.src_embedding(src[i,:,:].unsqueeze(1)) * math.sqrt(self.d_model) )
    src_embed_tensor = torch.stack(src_emb).squeeze()
    tgt = self.tgt_embed(tgt) * math.sqrt(self.d_model)
    src_embed_tensor = self.pos_encod(src_embed_tensor)
    tgt = self.tgt_encod(tgt)
    src_embed_tensor = self.flat(src_embed_tensor)
    memory = self.transformer_encoder(src_embed_tensor.permute(0,2,1))
    deco = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask, tgt_key_padding_mask=pad_mask)
    output = self.lin_soft_stack(deco)
    return output

In [ ]:
#d_mod = n_channels = 256 (embedding dimensions)
model = TransformerModel(n_tok, 256,1024).to(device)
model = torch.nn.DataParallel(model)


In [ ]:
loss_list_train = []
loss_list_test = []
acc_list_train = []
acc_list_test = []

#Define train and test loop
def train_loop(dataloader, model, loss_fn, optimizer):
  model.train()
  size = len(dataloader.dataset)
  for batch, (X, y, paths) in enumerate(dataloader):
    size, correct = 0, 0
    X, y = X.to(device), y.to(device) # X.size() == (batch_size, height, width); y.size() == (batch_size, seq_length)
    tgt_mask = generate_square_subsequent_mask(y.size(1))
    pad_mask = create_pad_mask(y, 3)

    #Compute prediction and loss
    pred = model(X, y, tgt_mask, pad_mask) # pred.size() == (batch_size, seq_length, d_mod)
    loss = loss_fn(pred.permute(0,2,1), y)
    acc_pack = batch_accuracy_calculate(pred.argmax(2), y) # pred is maximized along d_mod dimension allowing comparison with y
    correct += acc_pack[0]
    size += acc_pack[1]
    correct /= size
    acc_list_train.append(correct) #Take note of the single batch accuracy

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_list_train.append(loss.item()) #
    if batch % 10 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Funzione di loss: {loss:>7f} [{current:>5d}/{size:>5d}/8300]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
      for X, y, paths in dataloader:
        batch_size, correct_batch =0, 0
        X, y = X.to(device), y.to(device)
        tgt_mask = generate_square_subsequent_mask(y.size(1))
        pad_mask = create_pad_mask(y, 3)
        #Compute prediction and loss
        pred = model(X, y, tgt_mask, pad_mask)
        loss = loss_fn(pred.permute(0,2,1), y)
        loss_list_test.append(loss.item()) #Take note of the single batch loss
        test_loss += loss_fn(pred.permute(0,2,1), y).item()
        acc_pack = batch_accuracy_calculate(pred.argmax(2), y) # pred is maximized along d_mod dimension allowing comparison with y
        correct += acc_pack[0]
        correct_batch += acc_pack[0]
        size += acc_pack[1]
        batch_size += acc_pack[1]
        correct_batch /= batch_size
        acc_list_test.append(correct_batch) #Take note of the batch accuracy


    test_loss /= num_batches
    correct /= size
    print(f"Errore test: \n Accuratezza: {(100*correct):>0.1f}%, Media loss: {test_loss:>8f} \n")

In [ ]:
#Definiamo la loss function e il metodo di minimizzazione
learning_rate = 1e-1
momentum = 0.9
batch_size = 64

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = momentum)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Funzione di loss: 4.605437 [    0/    9/8300]
Funzione di loss: 4.605071 [  640/   10/8300]
Funzione di loss: 4.603691 [ 1280/   15/8300]
Funzione di loss: 4.602553 [ 1920/   53/8300]
Funzione di loss: 4.597796 [ 2560/   15/8300]
Funzione di loss: 4.583098 [ 3200/   10/8300]
Funzione di loss: 4.546018 [ 3840/   13/8300]
Funzione di loss: 4.532440 [ 4480/   14/8300]
Funzione di loss: 4.523986 [ 5120/   15/8300]
Funzione di loss: 4.475482 [ 5760/   13/8300]
Funzione di loss: 4.445719 [ 6400/   15/8300]
Funzione di loss: 4.396024 [ 7040/   11/8300]
Funzione di loss: 4.361186 [ 7680/   10/8300]
Errore test: 
 Accuratezza: 57.8%, Media loss: 4.307397 

Epoch 2
-------------------------------
Funzione di loss: 4.322335 [    0/   19/8300]
Funzione di loss: 4.314105 [  640/    9/8300]
Funzione di loss: 4.296843 [ 1280/   17/8300]
Funzione di loss: 4.255865 [ 1920/   16/8300]
Funzione di loss: 4.193407 [ 2560/   17/8300]
Funzione di loss: 4.194396 [ 3200/   17/8300]
Funzione di loss: 4.192110 [

In [ ]:
lists_to_write = ['loss_list_train', 'loss_list_test', 'acc_list_train', 'acc_list_test']
data = {listname: globals()[listname] for listname in lists_to_write}
import json
with open('./data_res_lists.txt', 'w') as file:
    json.dump(data, file, indent=2)